# ReAct agent Pattern - LlamaIndex RAG Integration 

In [1]:
# Install required packages (if not already installed)
%pip install llama-index
%pip install llama-index-agent-openai  # For ReActAgent
%pip install llama-index-llms-huggingface-api
%pip install llama-index-embeddings-huggingface
%pip install llama-index-core
%pip install llama-index-llms-groq
%pip install llama-index-vector-stores-chroma
%pip install chromadb

  Using cached llama_index-0.14.8-py3-none-any.whl.metadata (13 kB)
  Using cached llama_index_cli-0.5.3-py3-none-any.whl.metadata (1.4 kB)
  Using cached llama_index_core-0.14.8-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_embeddings_openai-0.5.1-py3-none-any.whl.metadata (400 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.9.4-py3-none-any.whl.metadata (3.7 kB)
  Using cached llama_index_llms_openai-0.6.9-py3-none-any.whl.metadata (3.0 kB)
  Using cached llama_index_readers_file-0.5.5-py3-none-any.whl.metadata (5.7 kB)
  Using cached llama_index_readers_llama_parse-0.5.1-py3-none-any.whl.metadata (3.1 kB)
  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached aiosqlite-0.21.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached banks-2.2.0-py3-none-any.whl.metadata (12 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached deprecated-1.3.1-py2.py3-none-any.whl.metadata (5.9 kB)
  Using cached dirty

## Relevant imports and Groq Client

We start by importing all the libraries we'll be using in this tutorial as well as the Groq client.

In [1]:
import os
import re
import math
import json
from dotenv import load_dotenv

from pathlib import Path
from typing import List, Dict, Any
from IPython.display import display, Markdown
import time

# LlamaIndex core imports
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    Settings,
    load_index_from_storage
)
from llama_index.core.agent import ReActAgent, AgentWorkflow
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

# ChromaDB for vector storage
import chromadb

#from groq import Groq

from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.utils.extraction import extract_tag_content


# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
#GROQ_CLIENT = Groq()

> If you are not familiar with the `tool` decorator, chances are you are missed the previous tutorial about the Tool Pattern. Check the video [here](https://www.youtube.com/watch?v=ApoDzZP8_ck&t=671s&ab_channel=TheNeuralMaze).

In [2]:
# Set up paths
PROJECT_ROOT = Path("..")
VECTOR_DB_DIR = PROJECT_ROOT / "data" / "vector_db"
SAMPLE_DATA_DIR = PROJECT_ROOT / "resources" / "sample-datasets"

# Create necessary directories
VECTOR_DB_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 Project Root: {PROJECT_ROOT}")
print(f"💾 Vector DB Directory: {VECTOR_DB_DIR}")
print(f"📄 Sample Data Directory: {SAMPLE_DATA_DIR}")
print(f"\n✅ Paths configured successfully!")

📁 Project Root: ..
💾 Vector DB Directory: ../data/vector_db
📄 Sample Data Directory: ../resources/sample-datasets

✅ Paths configured successfully!


In [3]:

# Set up embedding model
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5",  # Lightweight, high-quality embedding model
    cache_folder=str(PROJECT_ROOT / "models")
)

# You can specify the model you want to use, e.g., "llama-3.3-70b-versatile"
# If you don't specify a model, it defaults to "mixtral-8x7b-32768"
llm = Groq(model="llama-3.3-70b-versatile")

# Configure global settings
Settings.embed_model = embed_model
Settings.llm = llm
Settings.chunk_size = 512
Settings.chunk_overlap = 50

print("✅ LlamaIndex settings configured:")
print(f"   - Embedding Model: BAAI/bge-small-en-v1.5")
print(f"   - LLM: Groq llama-3.3-70b-versatile")
print(f"   - Chunk Size: 512")
print(f"   - Chunk Overlap: 50")

# Now you can use it in your queries
#response = llm.complete("What is the distance between the Earth and the Moon?")
#print(response)

✅ LlamaIndex settings configured:
   - Embedding Model: BAAI/bge-small-en-v1.5
   - LLM: Groq llama-3.3-70b-versatile
   - Chunk Size: 512
   - Chunk Overlap: 50


## A System Prompt for the ReAct Loop

As we did with the Tool Pattern, we also need a System Prompt for the ReAct technique. This System Prompt is very similar, the difference is that it describes the ReAct loop, so that the LLM is aware of
the three operations it's allowed to use:

1. Thought: The LLM will think about which action to take
2. Action: The LLM will use a Tool to "act on the environment"
3. Observation: The LLM will observe the tool output and reflect on the next thing to do.

Another key difference from the Tool Pattern System Prompt is that we are going to enclose all the messages with tags, like these: <thought></thought>, <observation></observation>. We could implement the ReAct logic without these tags, but I found it eeasier for the LLM to understand the instructions this way.

Ok! So without further ado, there's the prompt!

In [82]:
# Define the System Prompt as a constant
REACT_SYSTEM_PROMPT = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
%s
</tools>

Example session:

<question>What's the parential leave duration according to the company HR policy?</question>
<thought>I need to get the parental leave from the internal company HR plicy related documents</thought>
<tool_call>{"name": "rag_query_engine","arguments": {"query": "parential leave duration"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {According to our HR policies, parental leave is 16 weeks. [Source: company_handbook.md]}</observation>

You then output:

<response>According to our HR policies, parental leave is 16 weeks. [Source: company_handbook.md]</response>

Additional constraints:

- Only provide information found in company documents
- If information is not found, explicitly state "I could not find..."
- Never make up or infer information not present in the documents
- For queries outside the knowledge base scope, politely decline
- If a query is too vague, ask for clarification before searching
- Always cite sources in the format [Source: document_name]
- If the user asks you something unrelated to any of the tools above, politly decline the question enclosing your answer with <response></response> tags.
"""

In [45]:
# Load existing vector index from Phase 1
print("🔄 Loading existing vector index from Phase 1...")

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path=str(VECTOR_DB_DIR))
collection_name = "internal_knowledge_base"

# Load the collection
try:
    chroma_collection = chroma_client.get_collection(name=collection_name)
    print(f"✅ Found existing collection: {collection_name}")
    print(f"   Total vectors: {chroma_collection.count()}")
except Exception as e:
    print(f"❌ Error loading collection: {e}")
    print("   Please run Phase 1 notebook first to create the vector index.")
    raise

# Create ChromaVectorStore wrapper
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir=str(VECTOR_DB_DIR))

# Load the index
try:
    index = load_index_from_storage(storage_context)
    print("✅ Vector index loaded successfully!")
except Exception as e:
    print(f"❌ Error loading index: {e}")
    raise

🔄 Loading existing vector index from Phase 1...
✅ Found existing collection: internal_knowledge_base
   Total vectors: 3
✅ Vector index loaded successfully!


### Defining the Tools

Let's build an RAG tool that involves the use of a rag_query_engine tool.

In [ ]:
# Create QueryEngineTool from the vector index
print("🔧 Setting up QueryEngine Tool...")

# Create query engine with optimized settings for agent use
query_engine = index.as_query_engine(
    similarity_top_k=5,      # Retrieve top 5 most similar chunks
    response_mode="compact", # Concatenate chunks and generate single response
    streaming=False
)

@tool
def rag_query_engine(query: str) -> str:
    """
    A tool to query the knowledge base containing internal company documents. 

    Args:
        query (str): The query string to search the knowledge base.

    Returns:
        str: The response from the knowledge base along with citation sources.
        example: According to our HR policies, parental leave is 16 weeks. [Sources: company_handbook.md].
    """

    citations = []
    response = query_engine.query(query)

    if response and hasattr(response, "source_nodes"):
        citations = [node.node.metadata.get('file_name', 'Unknown') for node in response.source_nodes]

    final_answer = str(response) + f"\n\n[Sources: {', '.join(citations)}" + "]"
    return final_answer


available_tools = {
    "rag_query_engine": rag_query_engine
}

🔧 Setting up QueryEngine Tool...


Remember that the `@tool` operator allows us to convert a Python function into a `Tool` automatically. We cana check that very easily with some of the functions above.

In [84]:
print("Tool name: ", rag_query_engine.name)
print("Tool signature: ", rag_query_engine.fn_signature)

Tool name:  rag_query_engine
Tool signature:  {"name": "rag_query_engine", "description": "\n    A tool to query the knowledge base containing internal company documents. \n\n    Args:\n        query (str): The query string to search the knowledge base.\n\n    Returns:\n        str: The response from the knowledge base along with citation sources.\n        example: According to our HR policies, parental leave is 16 weeks. [Sources: company_handbook.md].\n    ", "parameters": {"properties": {"query": {"type": "str"}}}}


### Adding the Tools signature to the System Prompt

Now, we just concatenate the tools signature and add them to the System Prompt.

In [85]:
tools_signature = rag_query_engine.fn_signature 

In [78]:
print(tools_signature)

{"name": "rag_query_engine", "description": "\n    A tool to query the knowledge base containing internal company documents. \n\n    Args:\n        query (str): The query string to search the knowledge base.\n\n    Returns:\n        str: The response from the knowledge base along with citation sources.\n    ", "parameters": {"properties": {"query": {"type": "str"}}}}


In [86]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature

In [87]:
print(REACT_SYSTEM_PROMPT)


You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
{"name": "rag_query_engine", "description": "\n    A tool to query the knowledge base containing internal company documents. \n\n    Args:\n        query (str): The query string to search the knowledge base.\n\n    Returns:\n        str: The response from the knowledge base alon

## Using the `agentic_patterns` library 

In [88]:
from agentic_patterns.planning_pattern.react_agent import ReactAgent

In [ ]:
# TODO: configure the Groq LLM to use a structured output schema to correctly return a response that is appended with citations info 
#       (using [source: xxx.md, ...] format) at the end 
agent = ReactAgent(tools=[rag_query_engine])

In [90]:
agent.run(user_msg="How do I set up the local dev environment for project Nexus?")
#agent.run(user_msg="How do I set up the local dev environment for WordPress project?")
#agent.run(user_msg="Tell me the weather?")


Thought: I need to query the knowledge base to find information about setting up the local dev environment for project Nexus.

Using Tool: rag_query_engine

Tool call dict: 
{'name': 'rag_query_engine', 'arguments': {'query': 'setting up local dev environment for project Nexus'}, 'id': 0}

Tool result: 
To set up your local development environment for Project Nexus, follow these steps:

1. Install Git by following the instructions on the official Git website.
2. Download and install Docker Desktop from the official Docker website.
3. Install Python 3.9, as it is required for Project Nexus due to a dependency issue.
4. Install Node.js version 16 from the official Node.js website.
5. Install the internal 'Nexus' library by running the command `pip install nexus-library`.

After setting up your local environment, you can request access to cloud resources for Project Nexus using the `CloudProvisioner` tool. However, note that the command provided in the Project Nexus guide is outdated. In

"To set up your local development environment for Project Nexus, follow these steps: \n1. Install Git by following the instructions on the official Git website.\n2. Download and install Docker Desktop from the official Docker website.\n3. Install Python 3.9, as it is required for Project Nexus due to a dependency issue.\n4. Install Node.js version 16 from the official Node.js website.\n5. Install the internal 'Nexus' library by running the command `pip install nexus-library`.\nAfter setting up your local environment, you can request access to cloud resources for Project Nexus using the `CloudProvisioner` tool with the command `cprov request --role=developer --project=general`."

---

ReAct Agent - LlamaIndex Integration working as expected! 🚀🚀🚀🚀